<a href="https://colab.research.google.com/github/gopalm-ai/QOSF_2022/blob/master/QOSF_Task2_Qiskit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QOSF Task 2

1.Encode the following files in a quantum circuit - mock_train_set.csv and mock_test_set.csv - in at least two different ways (these could be basis, angle, amplitude, kernel or random encoding).

2.Design a variational quantum circuit for each of the encodings, uses the column 4 as the target, this is a binary class 0 and 1.

3.You must use the data from column0 to column3 for your proposed classifier.

4.Consider the ansatz you are going to design as a layer and find out how many layers are necessary to reach the best performance.

## Install required libraries and dependencies:

In [36]:
! pip3 install qiskit
! pip3 install qiskit_machine_learning
! pip3 install pandas
! pip3 install numpy
! pip3 install scipy

## Import required libraries and dependencies

In [37]:
# Import PennyLane, wrapped NumPy version provided by PennyLane, and an optimizer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from qiskit import *
from qiskit.utils import *
from qiskit.opflow import *
from qiskit.circuit import *
from qiskit.circuit.library import *
from qiskit.algorithms.optimizers import *
from qiskit_machine_learning.neural_networks import TwoLayerQNN, CircuitQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.exceptions import QiskitMachineLearningError
from typing import Union
import time as t
from math import pi

In [38]:
pwd

'/content/drive/MyDrive/QOSF_2022'

In [39]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
cd drive/MyDrive/QOSF_2022/

[Errno 2] No such file or directory: 'drive/MyDrive/QOSF_2022/'
/content/drive/MyDrive/QOSF_2022


## Load training dataset "mock_train_set.csv" and convert to pandas dataframe 

In [41]:
train_data= pd.read_csv("mock_train_set.csv", delimiter=',')
train_data

,0,1,2,3,4
0,2789.26,1000.0,10.0,20.0,0.0
1,4040.01,1000000.0,1.0,1.0,1.0
2,2931.20,10000.0,10000.0,40.0,1.0
3,3896.54,10000.0,100000.0,30.0,1.0
4,982.06,100.0,1000.0,75.0,0.0
...,...,...,...,...,...
295,1464.52,1000000.0,10000.0,45.0,1.0
296,3493.66,1000000.0,100000.0,85.0,1.0
297,4182.81,1.0,1.0,65.0,0.0
298,3113.75,10000.0,100.0,1.0,1.0


## Load test dataset "mock_test_set.csv" and convert to pandas dataframe 

In [42]:
test_data= pd.read_csv("mock_test_set.csv", delimiter=',')
test_data

,0,1,2,3,4
0,2988.55,10000.0,10000.0,75.0,1.0
1,3413.80,1.0,100.0,90.0,0.0
2,3891.52,1.0,1.0,5.0,0.0
3,4514.99,10000.0,1000000.0,25.0,1.0
4,752.29,10.0,10.0,90.0,0.0
...,...,...,...,...,...
115,3155.94,10000.0,10000.0,35.0,1.0
116,2795.04,1.0,10.0,45.0,0.0
117,3702.48,1.0,1.0,20.0,0.0
118,2790.57,1.0,1.0,40.0,0.0


In [43]:
train_data.columns

Index(['0', '1', '2', '3', '4'], dtype='object')

In [44]:
train_data["0"].shape

(300,)

In [45]:
test_data["0"].shape

(120,)

## Normalize training data and convert to numpy array

In [46]:
train_data_norm = (train_data-train_data.min())/(train_data.max()-train_data.min())

In [47]:
train_data_norm

,0,1,2,3,4
0,0.554566,0.000999,0.000009,0.213483,0.0
1,0.807077,1.000000,0.000000,0.000000,1.0
2,0.583222,0.009999,0.009999,0.438202,1.0
3,0.778112,0.009999,0.099999,0.325843,1.0
4,0.189715,0.000099,0.000999,0.831461,0.0
...,...,...,...,...,...
295,0.287118,1.000000,0.009999,0.494382,1.0
296,0.696775,1.000000,0.099999,0.943820,1.0
297,0.835906,0.000000,0.000000,0.719101,0.0
298,0.620076,0.009999,0.000099,0.000000,1.0


In [48]:
train_data_array = train_data_norm.to_numpy()
train_data_array

array([[5.54566084e-01, 9.99000999e-04, 9.00000900e-06, 2.13483146e-01,
        0.00000000e+00],
       [8.07076552e-01, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00],
       [5.83221959e-01, 9.99901000e-03, 9.99901000e-03, 4.38202247e-01,
        1.00000000e+00],
       ...,
       [8.35906050e-01, 0.00000000e+00, 0.00000000e+00, 7.19101124e-01,
        0.00000000e+00],
       [6.20076475e-01, 9.99901000e-03, 9.90000990e-05, 0.00000000e+00,
        1.00000000e+00],
       [9.13584185e-01, 9.99901000e-03, 9.99991000e-02, 1.00000000e+00,
        1.00000000e+00]])

In [49]:
test_data_norm = (test_data-test_data.min())/(test_data.max()-test_data.min())

In [50]:
test_data_array = test_data_norm.to_numpy()
test_data_array

array([[6.05707520e-01, 9.99901000e-03, 9.99901000e-03, 8.31460674e-01,
        1.00000000e+00],
       [6.92819816e-01, 0.00000000e+00, 9.90000990e-05, 1.00000000e+00,
        0.00000000e+00],
       [7.90680572e-01, 0.00000000e+00, 0.00000000e+00, 4.49438202e-02,
        0.00000000e+00],
       [9.18398158e-01, 9.99901000e-03, 1.00000000e+00, 2.69662921e-01,
        1.00000000e+00],
       [1.47610532e-01, 9.00000900e-06, 9.00000900e-06, 1.00000000e+00,
        0.00000000e+00],
       [2.81139701e-01, 9.99901000e-03, 9.99000999e-04, 4.49438202e-02,
        1.00000000e+00],
       [6.88077548e-01, 1.00000000e+00, 9.99901000e-03, 4.49438202e-02,
        1.00000000e+00],
       [5.30410949e-01, 0.00000000e+00, 9.00000900e-06, 8.31460674e-01,
        0.00000000e+00],
       [3.56950445e-02, 9.00000900e-06, 9.90000990e-05, 1.57303371e-01,
        0.00000000e+00],
       [2.99908023e-01, 9.99901000e-03, 9.99901000e-03, 8.87640449e-01,
        1.00000000e+00],
       [4.29938361e-02, 9.9990

# TwoLayerQNN

In [51]:
x0 = Parameter('x0')
x1 = Parameter('x1')
x2 = Parameter('x2')
x3 = Parameter('x3')

qc = QuantumCircuit(4)
qc.rx(x0, 0)
qc.rx(x1, 1)
qc.rx(x2, 2)
qc.rx(x3, 3)

qc.ry(np.pi/4, 0)
qc.ry(np.pi/4, 1)
qc.ry(np.pi/4, 2)
qc.ry(np.pi/4, 3)

qc.rz(np.pi/4, 0)
qc.rz(np.pi/4, 1)
qc.rz(np.pi/4, 2)
qc.rz(np.pi/4, 3)

feature_map= [qc.bind_parameters({x0: x0_val}) for x0_val in train_data_array[:, 0]]
feature_map= [qc.bind_parameters({x1: x1_val}) for x1_val in train_data_array[:, 1]]
feature_map= [qc.bind_parameters({x2: x2_val}) for x2_val in train_data_array[:, 2]]
feature_map= [qc.bind_parameters({x3: x3_val}) for x3_val in train_data_array[:, 3]]

qc.draw()

┌────────┐┌─────────┐┌─────────┐
q_0: ┤ Rx(x0) ├┤ Ry(π/4) ├┤ Rz(π/4) ├
     ├────────┤├─────────┤├─────────┤
q_1: ┤ Rx(x1) ├┤ Ry(π/4) ├┤ Rz(π/4) ├
     ├────────┤├─────────┤├─────────┤
q_2: ┤ Rx(x2) ├┤ Ry(π/4) ├┤ Rz(π/4) ├
     ├────────┤├─────────┤├─────────┤
q_3: ┤ Rx(x3) ├┤ Ry(π/4) ├┤ Rz(π/4) ├
     └────────┘└─────────┘└─────────┘

## Define ansatz (TwoLocal - RX, RZ, CRX) and entangler map 

In [52]:
dim = 4
entangler_map = [(0, 1), (0, 2), (0, 3), (1, 0), (1, 2), (1, 3), (2, 0), (2, 1), (2, 3), (3, 0),  (3, 1), (3, 2)]
entangler_map2 = []
ansatz = TwoLocal(dim, ['rx', 'rz'], 'crx', [entangler_map, entangler_map2], reps=3, skip_final_rotation_layer= True, insert_barriers=True)

In [53]:
quantum_instance = QuantumInstance(Aer.get_backend('aer_simulator'), shots=1024)
algorithm_globals.random_seed = 42

In [55]:
from qiskit_machine_learning.neural_networks import TwoLayerQNN
num_inputs = 4
opflow_qnn = TwoLayerQNN(num_inputs, feature_map = qc, ansatz= ansatz, quantum_instance=quantum_instance)
print(opflow_qnn.circuit)

     ┌──────────────────────────────┐»
q_0: ┤0                             ├»
     │                              │»
q_1: ┤1                             ├»
     │  circuit-254321(x0,x1,x2,x3) │»
q_2: ┤2                             ├»
     │                              │»
q_3: ┤3                             ├»
     └──────────────────────────────┘»
«     ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                                                                                                                                                                                                                 ├
«     │                                            

In [56]:
weights = algorithm_globals.random.random(opflow_qnn.num_weights)
# QNN maps inputs to [-1, +1]
opflow_qnn.forward(train_data_array[0, 0:4], weights)

array([[0.1796875]])

In [57]:
opflow_qnn.backward(train_data_array[0, 0:4], weights)

(None, array([[[-0.03125   , -0.14160156,  0.08203125,  0.00976563,
           0.04492188,  0.02832031, -0.00585938, -0.04589844,
          -0.24951172,  0.09960938,  0.07470703,  0.03076172,
           0.02246094, -0.01416016,  0.06494141,  0.00390625,
           0.00537109,  0.06005859,  0.01416016,  0.01464844,
          -0.02539062, -0.12011719,  0.04492188,  0.04101563,
           0.10058594,  0.09765625,  0.07128906, -0.01367188,
           0.12011719, -0.19628906,  0.01855469,  0.00878906,
           0.09277344,  0.07910156,  0.08300781,  0.06445312,
          -0.14550781,  0.04638672,  0.10253906,  0.0625    ,
           0.00976562,  0.00732422,  0.07373047, -0.12402344,
           0.01708984,  0.07177734, -0.16064453,  0.01220703]]]))

## Callback function


In [58]:
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [59]:
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit.algorithms.optimizers import COBYLA
# construct neural network classifier
opflow_classifier = NeuralNetworkClassifier(opflow_qnn, optimizer=COBYLA())

## Test Classifier

In [60]:
objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)

training_labels = 2 * train_data_array[:, 4] - 1
test_labels = 2 * test_data_array[:, 4] - 1

# fit classifier to data
opflow_classifier.fit(train_data_array[:, 0:4] , training_labels)

# return to default figsize
plt.rcParams["figure.figsize"] = (6, 4)

# score classifier
qnn_score = opflow_classifier.score(test_data_array[:, 0:4], test_labels)

with open('accuracy_vals_adhoc_zzfeaturemap.txt', 'a') as f:
    f.write(str(qnn_score)+' ') 
    np.save('accuracy_vals_adhoc_zzfeaturemap.npy', qnn_score)
print(f'QNN classification test score: {qnn_score}')

# predict labels
predicted = opflow_classifier.predict(test_data_array[:, 0:4])

QNN classification test score: 0.8666666666666667


# TwoLayerQNN with different ansatz

## Define ansatz (TwoLocal - RZ, RX, CRZ) and entangler map 

In [62]:
dim = 4
entangler_map = [(0, 1), (0, 2), (0, 3), (1, 0), (1, 2), (1, 3), (2, 0), (2, 1), (2, 3), (3, 0),  (3, 1), (3, 2)]
entangler_map2 = []
ansatz = TwoLocal(dim, ['rz', 'rx'], 'crz', [entangler_map, entangler_map2], reps=3, skip_final_rotation_layer= True, insert_barriers=True)

In [63]:
quantum_instance = QuantumInstance(Aer.get_backend('aer_simulator'), shots=1024)
algorithm_globals.random_seed = 42

In [64]:
from qiskit_machine_learning.neural_networks import TwoLayerQNN
num_inputs=4
opflow_qnn = TwoLayerQNN(num_inputs, feature_map = qc2, ansatz= ansatz, quantum_instance=quantum_instance)
print(opflow_qnn.circuit)

     ┌───────────────────────────┐»
q_0: ┤0                          ├»
     │                           │»
q_1: ┤1                          ├»
     │  circuit-989(x0,x1,x2,x3) │»
q_2: ┤2                          ├»
     │                           │»
q_3: ┤3                          ├»
     └───────────────────────────┘»
«     ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                                                                                                                                                                                                                 ├
«     │                                                                       

In [65]:
weights2 = algorithm_globals.random.random(opflow_qnn.num_weights)
# QNN maps inputs to [-1, +1]
opflow_qnn.forward(train_data_array[0, 0:4], weights2)

array([[0.38671875]])

In [66]:
opflow_qnn.backward(train_data_array[0, 0:4], weights2)

(None, array([[[ 0.18847656,  0.15917969, -0.37207031,  0.05078125,
          -0.16503906, -0.36621094, -0.28320313, -0.07910156,
          -0.06591797, -0.10205078, -0.08886719, -0.00976562,
          -0.04345703, -0.03710938, -0.00292969,  0.01416016,
           0.03808594, -0.01171875, -0.01269531, -0.01367188,
           0.18554687,  0.08398438, -0.02441406,  0.01660156,
          -0.12792969, -0.40527344, -0.42480469, -0.05566406,
           0.09082031,  0.06933594,  0.10351562,  0.00488281,
          -0.09960938, -0.32421875, -0.4609375 ,  0.03808594,
          -0.00146484, -0.00097656,  0.01171875, -0.00683594,
           0.02099609,  0.01025391, -0.01416016,  0.03271484,
           0.02294922, -0.00146484, -0.00097656,  0.02246094]]]))

## Callback Function

In [67]:
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [68]:
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit.algorithms.optimizers import COBYLA
# construct neural network classifier
opflow_classifier = NeuralNetworkClassifier(opflow_qnn, optimizer=COBYLA())

## Test Classifier

In [69]:
objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)

training_labels = 2 * train_data_array[:, 4] - 1
test_labels = 2 * test_data_array[:, 4] - 1

# fit classifier to data
opflow_classifier.fit(train_data_array[:, 0:4] , training_labels)

# return to default figsize
plt.rcParams["figure.figsize"] = (6, 4)

# score classifier
qnn_score = opflow_classifier.score(test_data_array[:, 0:4], test_labels)

with open('accuracy_vals_adhoc_zzfeaturemap.txt', 'a') as f:
    f.write(str(qnn_score)+' ') 
    np.save('accuracy_vals_adhoc_zzfeaturemap.npy', qnn_score)
print(f'QNN classification test score: {qnn_score}')

# predict labels
predicted = opflow_classifier.predict(test_data_array[:, 0:4])

QNN classification test score: 0.7833333333333333


# References:

1) Santanu Ganguly, *Quantum Machine Learning: An Applied Approach*, Apress, 2021.

2) Santanu Pattanayak, *Quantum Machine Learning with Python*, Apress, 2021.

3) PennyLane Demos: https://pennylane.ai/qml/demos/

4) Tensorflow Quantum: https://www.tensorflow.org/quantum/tutorials/

5) Qiskit Machine Learning: https://medium.com/qiskit/introducing-qiskit-machine-learning-5f06b6597526

6) Qiskit Textbook (beta): https://qiskit.org/textbook-beta/


